# WhatsApp group Chat data preprocessing

We wanted to incorporate questions and answers from a Whatsapp group for support of new students.  


In [ ]:
import re, json
from datetime import datetime, timedelta
import pandas as pd
from duckdb import sql as sqldf

from plotly import express as px
from plotly import graph_objs as go

## Export the WhatsApp chat data
You can find instructions anywhere on the internet  
Save it as _chat.txt in the same directory


## Load WhatsApp export
For the actual competition we've used an actual data from one of the support groups in our college.  
But we don't want to share this data with the world, so we're not publishing it.

There are similar datasets available on kaggle, so we will use one of them for the demonstration:  
[https://www.kaggle.com/datasets/kamyamehra/chat-dataset]()  

A row in this raw caht dump generally looks like: 

`[24/04/24, 7:54:07 PM] ~ Jigyasu: Hello, I am Jigyasu from NIT Agartala`

Below are the regexes needed to parse this dump.  
Be advised that different versions of whatsapp have slightly different row format. Additionally the datetime format seems to depend on the locale defined in the smartphone of the person who exported it.  
This means that you will need to adapt these regexes to your particular dump.


In [ ]:
with open("_chat.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

# ambiguous_chars_pattern = r'[\u202b\u202c\u200f]' #rtl languages
bad_space_patt = r'[\u202f]'
bad_empty_patt = r'[\u202a\u202b\u202c\u200e\u200f]'


date_patt = r'([0-9]{1,2}\/[0-9]{1,2}\/[0-9]{2,4})'
time_patt = r'([0-9]{1,2}:[0-9]{2}:[0-9]{2}\s*[AaPp][Mm])'
auth_patt = r'(?:~)?\s*([^:]+)'
text_patt = r'(.*)'
full_patt = f'^\[{date_patt},\s*{time_patt}\]\s*{auth_patt}:\s*{text_patt}$'
line_re = re.compile(full_patt)

print(full_patt)

In [ ]:
dt_format = "%d/%m/%y %H:%M:%S %p"

rows = []
row = {}

for raw_line in raw_text.splitlines():
    raw_line = re.sub(bad_space_patt, ' ', raw_line) 
    raw_line = re.sub(bad_empty_patt, '', raw_line) 
    m = line_re.match(raw_line)
    if m:
        rows.append(row)       
        
        date_str, time_str, author, text = m.groups()
        dt = datetime.strptime(f"{date_str} {time_str}", dt_format)
        row = {"timestamp": dt, "author": author.strip(), "text": text.strip()}
        
    else:
        row["text"] += '\n' + raw_line
print(len(rows))
rows[:3]

In [ ]:
df = pd.DataFrame(rows[1:]).sort_values("timestamp").reset_index(drop=True)

In [ ]:
df[30:].head(30)

In [ ]:
df['date'] = df["timestamp"].dt.date
df['time'] = df["timestamp"].dt.time
df['daytime'] = df.time.apply(lambda dt : dt.hour*60*60 + dt.minute*60 + dt.second)


In [ ]:
px.histogram(df, x = "date", nbins=df.date.nunique())

## Detect staff 
In the original data, we know who the staff are.  

For this public data we'll just assume that several most active users are the staff.

In [ ]:
q = r"""SELECT author, count(text) as cnt

        FROM df
        
        group by author
        order by cnt desc
        ;""" 

userstats = sqldf(q).df()
userstats

In [ ]:
px.bar(userstats, x = userstats.index, y = "cnt", hover_data=["author"])

In [ ]:
staff_names = set(userstats[:4].author)
staff_names

In [ ]:
staff_names = {'Amritaa Sethi Mam Gfg','Aman Sir Gfg Interview', 'Nikhil Sharma', '.', 'Aadil Latif'}

df["role"] = df["author"].apply(lambda a: "staff" if a in staff_names else "student")


In [ ]:
df

## Anonymize the data 
We will replace the actial names with ids 

In [ ]:
authUnq = pd.DataFrame(df.author.unique()).reset_index()
authUnq.columns = ["Id","name"]
authUnq.Id = 1000 + authUnq.Id
authUnq

In [ ]:
px.histogram(df, x = 'daytime', color = 'role', nbins=600)

## Clean up trash
Some messages are just trash. Let's find those that repeat the most.

In [ ]:
q = r"""SELECT text, count(text) as cnt

        FROM df
        -- where text like '%omitted%'
        group by text
        order by cnt desc
        ;""" 

lal = sqldf(q).df()
print(lal.shape)
lal.head(30)

Not all of these are trash. We will classify them into several groups.

In [ ]:
dels = list(lal.loc[[0,1,5,6,7,8,11,13,15]].text)
dels

In [ ]:
dels += ['Thanks', 'Ty', 'video omitted']

urlfile = ["https://", ".pdf", ".docx", ".png"]

dels_like = ['document omitted','This message was deleted by admin']

joined = ["added ~ ", "removed ~ ", "removed +"]

when =  [f"WHEN text = '{t}' THEN 'deleted'" for t in dels]
when += [f"WHEN text LIKE '%{t}%' THEN 'deleted'" for t in dels_like]
when += [f"WHEN text LIKE '%{t}%' THEN 'urlfile'" for t in urlfile]
when += [f"WHEN text LIKE '%{t}%' THEN 'joined'" for t in joined]


when = "\n\t".join(when)
print(when)

In [ ]:
dels

In [ ]:
basecols = ", ".join(df.columns)
basecols

In [ ]:
q = f"""SELECT {basecols}, 'user' || cast(au.id as string) as authorAnon,
    CASE
        {when}
        ELSE role
    END as class      
    
    FROM df
    JOIN authUnq au on df.author = au.name
        
        ;""" 
print(q)

In [ ]:
df = sqldf(q).df()
df

In [ ]:
px.histogram(df, x = 'class')

## Visualaization
It may be interesting to view the whole data in a temporal perspective, where:
- x axis is time
- y axis are the individual users
- each dot represents a single message
- dot color is message's class

By using an interactive plotting library such as plotly, we may dive deeper into this ocean of points and perhaps discover some interesting patterns.

In [ ]:
fig = px.scatter(df, x = "timestamp", y = "author", color = 'class', 
                 height=800, hover_data=["text"],  )
fig.update_traces(marker_size = 3)

Another interesting projection is the time vs. time-of-day:

 * Note: there was some problem with properly presenting time-of-day as HH:MM:SS, so I decided to present it as total seconds since beginning of the day.

In [ ]:
fig = px.scatter(df, x = "timestamp", y = "daytime", color = 'class', 
                 height=600, hover_data=["author","text"]  )

fig.update_traces(marker_size = 4)
fig.update_layout(yaxis_tickformat='%H:%M:%S')

If we zoom x axis to a resolution of one day:

![](cluster.png)

We will notice that the messages are oftern grouped together in separate time spans. We surmize that these are separate threads each discussing an individual topic. We think that this may be used to separate the whole dataset into a series of isolated discussions for use in a RAG.

## Filtering


In [ ]:
q = f"""SELECT *

        FROM df
        where class in ('student','staff','urlfile')
        ;""" 
flt = sqldf(q).df()
flt.shape

In [ ]:
flt

In [ ]:
fig = px.scatter(flt, x = "timestamp", y = "daytime", color = 'role', 
                 height=800, hover_data=["author","text"] )
fig.update_traces(marker_size = 3)
fig.update_layout(yaxis_tickformat='%H:%M:%S')

In [ ]:
df.text.nunique()

For now we will just remove only those messages that are definitely trash.  
For the conversations/dialogues separation we'll use simpler approach: just separate them by day. 

In [ ]:
q = f"""
with s1 as 
(SELECT date, group_concat(time || ' ' || authorAnon || ': ' || text, '<hr>') as html_anon,
                group_concat(time || ' ' || author || ': ' || text, '<hr>') as html,
group_concat( authorAnon || ': ' || text, '\n') as txt

        FROM flt
        GROUP by date
)
select * 

from s1
order by date
        ;""" 

dialogues = sqldf(q).df()
dialogues.head()

## Output

In [ ]:

def save_df_as_pretty_html(df, filename="output.html", index=True):
    pd.set_option("display.max_colwidth", None)
    # Convert newlines to <br> for HTML
    df_html_ready = df.copy()
    for col in df_html_ready.columns:
        df_html_ready[col] = df_html_ready[col].astype(str).str.replace('\n', '<br>', regex=False)

    # Generate styled HTML
    html = df_html_ready.to_html(
        escape=False,  # Needed to render <br>
        index=index,
        border=0,
        classes="styled-table"
    )

    # Add CSS styling
    style = """
    <style>
    .styled-table {
        border-collapse: collapse;
        margin: 25px 0;
        font-size: 16px;
        font-family: Arial, sans-serif;
        width: 100%;
        table-layout: auto; /* ✅ Let browser fit naturally */
    }
    .styled-table th, .styled-table td {
        border: 1px solid #dddddd;
        padding: 10px;
        vertical-align: top;
        text-align: left;
        overflow-wrap: break-word; /* ✅ Break inside words */
        white-space: pre-wrap; /* ✅ Honor \\n linebreaks */
    }
    .styled-table td {
        max-width: 600px; /* ✅ Avoid huge dream fields expanding table */
    }
    .styled-table th {
        background-color: #f2f2f2;
    }
    </style>
    """

    # Write full HTML document
    with open(filename, "w", encoding="utf-8") as f:
        f.write(f"<!DOCTYPE html><html><head>{style}</head><body>{html}</body></html>")

    print(f"✅ HTML table saved to: {filename}")


And save it in 2 formats: 
- html for convenient viewing
- json for exporting to RAG

In [ ]:
save_df_as_pretty_html(dialogues[['date','html']], 'dialogues.html')

In [ ]:
dialogues['txt'].to_json('dialogues.json', force_ascii=False, indent=2, orient='records')

That's all for now